# MSCA 31009 Machine Learning & Predictive Analytics
## Final Project - Data Preprocessing
Shijia Huang

-----

In [1]:
# Import basic libraries
import sys
import os
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential, load_model

2023-05-14 22:01:17.050341: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [4]:
import multiprocessing as mp

num_processors = mp.cpu_count()
print(f'Available CPUs: {num_processors}')

Available CPUs: 12


## Read Data

### Games Data

In [5]:
# read games data
games_df = pd.read_csv('data/games.csv')
games_df.shape

(48756, 13)

In [6]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48756 entries, 0 to 48755
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   app_id          48756 non-null  int64  
 1   title           48756 non-null  object 
 2   date_release    48756 non-null  object 
 3   win             48756 non-null  bool   
 4   mac             48756 non-null  bool   
 5   linux           48756 non-null  bool   
 6   rating          48756 non-null  object 
 7   positive_ratio  48756 non-null  int64  
 8   user_reviews    48756 non-null  int64  
 9   price_final     48756 non-null  float64
 10  price_original  48756 non-null  float64
 11  discount        48756 non-null  float64
 12  steam_deck      48756 non-null  bool   
dtypes: bool(4), float64(3), int64(3), object(3)
memory usage: 3.5+ MB


In [7]:
games_df.head()

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,10090,Call of Duty: World at War,2008-11-18,True,False,False,Very Positive,92,37039,19.99,19.99,0.0,True
1,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
2,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
3,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
4,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True


### Users Data

In [8]:
# read users data
users_df = pd.read_csv('data/users.csv')
users_df.shape

(6791175, 3)

In [9]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6791175 entries, 0 to 6791174
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   products  int64
 2   reviews   int64
dtypes: int64(3)
memory usage: 155.4 MB


In [10]:
users_df.head()

,user_id,products,reviews
0,6654146,156,1
1,4191643,329,4
2,2261155,176,2
3,2437528,98,2
4,2700117,144,3


### Reviews Data

In [11]:
# read reviews data
reviews_df = pd.read_csv('data/reviews.csv')
reviews_df.shape

(13207260, 8)

In [12]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13207260 entries, 0 to 13207259
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   app_id          int64  
 1   helpful         int64  
 2   funny           int64  
 3   date            object 
 4   is_recommended  bool   
 5   hours           float64
 6   user_id         int64  
 7   review_id       int64  
dtypes: bool(1), float64(1), int64(5), object(1)
memory usage: 717.9+ MB


In [13]:
reviews_df.head()

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
0,975370,0,0,2022-12-12,True,36.3,23221,0
1,304390,4,0,2017-02-17,False,11.5,1283,1
2,1085660,2,0,2019-11-17,True,336.5,105979,2
3,703080,0,0,2022-09-23,True,27.4,108133,3
4,526870,0,0,2021-01-10,True,7.9,10612,4


### Summary

In [14]:
pd.DataFrame([
    ["Reviews", len(reviews_df.index)],
    ["Users", len(users_df.index)],
    ["Games", len(games_df.index)]
], columns=["Dataframe", "Records"])

,Dataframe,Records
0,Reviews,13207260
1,Users,6791175
2,Games,48756


## Data Preprocessing

### Missing Values

In [15]:
# check for missing values
def pct_na(df: pd.DataFrame) -> float:
    """Calculates a percentage of missing values in the dataframe."""
    return np.count_nonzero(df.isnull().values) / (df.shape[0] * df.shape[1]) * 100.0

pd.DataFrame([
    ["Reviews", pct_na(reviews_df)],
    ["Users", pct_na(users_df)],
    ["Games", pct_na(games_df)]
], columns=["Dataframe", "Missing values (%)"])

,Dataframe,Missing values (%)
0,Reviews,0.0
1,Users,0.0
2,Games,0.0


### Games

In [16]:
# Check for duplicates
games_df['app_id'].unique().shape[0] == games_df.shape[0]

True

In [17]:
# Data types
games_df.dtypes

app_id              int64
title              object
date_release       object
win                  bool
mac                  bool
linux                bool
rating             object
positive_ratio      int64
user_reviews        int64
price_final       float64
price_original    float64
discount          float64
steam_deck           bool
dtype: object

In [18]:
# Basic statistics for numerical columns
games_df[['positive_ratio', 'user_reviews', 'price_final', 'price_original', 'discount']].describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
positive_ratio,48756.0,76.88,18.23,0.0,66.00,81.00,91.00,100.00
user_reviews,48756.0,1752.52,37996.73,10.0,20.00,50.00,212.00,6941137.00
price_final,48756.0,8.64,11.50,0.0,0.99,4.99,10.99,299.99
price_original,48756.0,9.17,11.79,0.0,1.99,4.99,12.99,299.99
discount,48756.0,5.06,17.94,0.0,0.00,0.00,0.00,90.00


In [19]:
# Boolean value counts for all boolean columns
games_df[['win', 'mac', 'linux', 'steam_deck']].apply(pd.Series.value_counts)

,win,mac,linux,steam_deck
False,751,36119,39979,NaN
True,48005,12637,8777,48756.0


In [20]:
# rating value counts
games_df['rating'].value_counts()

rating
Very Positive              12715
Positive                   12564
Mixed                      11808
Mostly Positive             8449
Mostly Negative             1794
Overwhelmingly Positive     1067
Negative                     290
Very Negative                 56
Overwhelmingly Negative       13
Name: count, dtype: int64

In [21]:
# Encoding rating column to numerical values
rating_map = {'Overwhelmingly Positive': 4,
              'Very Positive': 3,
              'Positive': 2,
              'Mostly Positive': 1,
              'Mixed': 0,
              'Mostly Negative': -1,
              'Negative': -2,
              'Very Negative': -3,
              'Overwhelmingly Negative': -4}

games_df['rating'] = games_df['rating'].map(rating_map)

In [22]:
# Phrase data types
games_df['date_release'] = pd.to_datetime(games_df['date_release'])

In [23]:
# Encode boolean values as 0 and 1
games_df[['win', 'mac', 'linux', 'steam_deck']] = games_df[['win', 'mac', 'linux', 'steam_deck']].astype(int)

In [24]:
games_df.head()

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,10090,Call of Duty: World at War,2008-11-18,1,0,0,3,92,37039,19.99,19.99,0.0,1
1,13500,Prince of Persia: Warrior Within™,2008-11-21,1,0,0,3,84,2199,9.99,9.99,0.0,1
2,22364,BRINK: Agents of Change,2011-08-03,1,0,0,2,85,21,2.99,2.99,0.0,1
3,113020,Monaco: What's Yours Is Mine,2013-04-24,1,1,1,3,92,3722,14.99,14.99,0.0,1
4,226560,Escape Dead Island,2014-11-18,1,0,0,0,61,873,14.99,14.99,0.0,1


In [25]:
games_df.dtypes

app_id                     int64
title                     object
date_release      datetime64[ns]
win                        int64
mac                        int64
linux                      int64
rating                     int64
positive_ratio             int64
user_reviews               int64
price_final              float64
price_original           float64
discount                 float64
steam_deck                 int64
dtype: object

In [26]:
# Save preprocessed data
games_df.to_csv('preprocessed/games.csv', index=False)

### Users

In [27]:
# Check for duplicates
users_df['user_id'].unique().shape[0] == users_df.shape[0]

True

In [28]:
# Data types
users_df.dtypes

user_id     int64
products    int64
reviews     int64
dtype: object

In [29]:
# Basic statistics for numerical columns
users_df[['products', 'reviews']].describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
products,6791175.0,132.24,261.30,0.0,23.0,62.0,150.0,29308.0
reviews,6791175.0,1.94,2.59,0.0,1.0,1.0,2.0,304.0


In [30]:
users_df.head()

,user_id,products,reviews
0,6654146,156,1
1,4191643,329,4
2,2261155,176,2
3,2437528,98,2
4,2700117,144,3


In [31]:
# Save preprocessed data
users_df.to_csv('preprocessed/users.csv', index=False)

### Reviews

In [32]:
# Check for duplicates
reviews_df['review_id'].unique().shape[0] == reviews_df.shape[0]

True

In [33]:
# Data types
reviews_df.dtypes

app_id              int64
helpful             int64
funny               int64
date               object
is_recommended       bool
hours             float64
user_id             int64
review_id           int64
dtype: object

In [34]:
# Basic statistics for numerical columns
reviews_df[['helpful', 'funny', 'hours']].describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
helpful,13207260.0,3.34,58.02,0.0,0.0,0.0,0.0,36212.0
funny,13207260.0,1.12,38.83,0.0,0.0,0.0,0.0,28088.0
hours,13207260.0,144.42,206.18,0.0,16.6,54.2,174.6,999.9


In [35]:
# Boolean value counts
reviews_df['is_recommended'].value_counts()

is_recommended
True     11291801
False     1915459
Name: count, dtype: int64

In [36]:
# Phrase data types
reviews_df['date'] = pd.to_datetime(reviews_df['date'])

In [37]:
# Encode boolean values as 0 and 1
reviews_df['is_recommended'] = reviews_df['is_recommended'].astype(int)

In [38]:
reviews_df.dtypes

app_id                     int64
helpful                    int64
funny                      int64
date              datetime64[ns]
is_recommended             int64
hours                    float64
user_id                    int64
review_id                  int64
dtype: object

In [39]:
reviews_df.head()

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
0,975370,0,0,2022-12-12,1,36.3,23221,0
1,304390,4,0,2017-02-17,0,11.5,1283,1
2,1085660,2,0,2019-11-17,1,336.5,105979,2
3,703080,0,0,2022-09-23,1,27.4,108133,3
4,526870,0,0,2021-01-10,1,7.9,10612,4


In [40]:
# Save preprocessed data
reviews_df.to_csv('preprocessed/reviews.csv', index=False)

### Merge Users and Reviews

In [41]:
# merge users and reviews dataframes
users_reviews_df = pd.merge(users_df, reviews_df, on='user_id', how='inner')
users_reviews_df.shape

(13207260, 10)

In [42]:
users_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13207260 entries, 0 to 13207259
Data columns (total 10 columns):
 #   Column          Dtype         
---  ------          -----         
 0   user_id         int64         
 1   products        int64         
 2   reviews         int64         
 3   app_id          int64         
 4   helpful         int64         
 5   funny           int64         
 6   date            datetime64[ns]
 7   is_recommended  int64         
 8   hours           float64       
 9   review_id       int64         
dtypes: datetime64[ns](1), float64(1), int64(8)
memory usage: 1007.6 MB


In [43]:
# check for if length of reviews is equal to length of reviews after merge
len(reviews_df.index) == len(users_reviews_df.index)

True

In [44]:
users_reviews_df.head()

,user_id,products,reviews,app_id,helpful,funny,date,is_recommended,hours,review_id
0,6654146,156,1,730,0,0,2020-09-13,1,515.9,389270
1,4191643,329,4,1259420,5,0,2021-05-18,1,78.9,3220984
2,4191643,329,4,271590,0,0,2022-04-11,1,397.3,3273691
3,4191643,329,4,1029690,0,0,2022-11-26,1,67.6,8177218
4,4191643,329,4,1517290,12,2,2021-11-23,0,1.2,12305209


In [45]:
# Save preprocessed data
users_reviews_df.to_csv('preprocessed/users_reviews.csv', index=False)